
## Lewis Hamilton - kierowca z siedmioma tytułami mistrza świata F1


### Lewis Carl Davidson Hamilton urodził się 7 stycznia 1985 roku w Stevenage.
### Karierę w F1 rozpoczął w 2007 roku, która trwa do dziś. 
### W ciągu 13 sezonów jakie brytyjczyk spędził do tej pory w F1 siedmiokrotnie został mistrzem świata. 
### W sezonie 2020 wyrównał rekord legendarnego Michaela Schumachera w liczbie zdobytych tytułów, który może pobić już w najbliższym sezonie 2021.

_Nieniejsze opracowanie ma na celu pokazanie Lewisa Hamiltona w liczbach_

In [1]:
# Importy
import time
import datetime

import numpy as np 
import pandas as pd
import scipy.stats as sp

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

In [2]:
# Wczytanie danych
races = pd.read_csv('races.csv')
status = pd.read_csv('status.csv')
drivers = pd.read_csv('drivers.csv')
results = pd.read_csv('results.csv')
constructors = pd.read_csv('constructors.csv')

In [3]:
# Imię i nazwisko kierowcy
concat_driver_name = lambda x: f"{x.forename} {x.surname}" 

drivers['driver'] = drivers.apply(concat_driver_name, axis=1)

In [186]:
# Przygotowanie tabeli zwycięzców F1
results_copy = results.set_index('raceId').copy()
races_copy = races.set_index('raceId').copy()

results_copy = results_copy.query("position == '1'")
results_copy['position'] = 1 # rzutowanie pozycji na int

results_cols = ['driverId', 'position']
races_cols = ['date']
drivers_cols = ['driver', 'driverId']

results_copy = results_copy[results_cols]
races_copy = races_copy[races_cols]
drivers_copy = drivers[drivers_cols]

f1_victories = results_copy.join(races_copy)
f1_victories = f1_victories.merge(drivers_copy, on='driverId', how='left')

# Skumulowana tabela zwycięzców Grand Prix
f1_victories = f1_victories.sort_values(by='date')

f1_victories['victories'] = f1_victories.groupby(['driverId']).cumsum() 

# Wybór najlepszej piątki
f1_biggest_winners = f1_victories.groupby('driverId').victories.nlargest(1).sort_values(ascending=False).head(5)
f1_biggest_winners_ids = [driver for driver, race in f1_biggest_winners.index]

# Gotowy zbiór
f1_victories_biggest_winners = f1_victories.query(f"driverId == {f1_biggest_winners_ids}")


In [230]:
# Przygotowanie wykresu

cols = ['date', 'driver', 'victories']
winner_drivers = f1_victories_biggest_winners.driver.unique()

colors = {
    'Alain Prost': '#500000', 
    'Ayrton Senna': '#ffffff', 
    'Michael Schumacher': '#f71120',
    'Sebastian Vettel': '#10428e',
    'Lewis Hamilton': '#b803ff'
}

#'Lewis Hamilton': '#e6e6e6'
winners_history = pd.DataFrame()

# Stworzenie tablicy, obrazującej zwycięstwa 5 kierowców z największą liczbą zwycieństw na osi czasu
for driver in winner_drivers:
    
    # Zwycięstwa bierzącego kierowcy
    driver_history = f1_victories_biggest_winners.query(f"driver == '{driver}'")[cols]
    
    # Lista pozostałych kierowców
    other_drivers = winner_drivers[winner_drivers != driver]
    other_drivers = list(other_drivers)
    
    # Zwycięstwa pozostałych kierowców
    other_driver_history = f1_victories_biggest_winners.query(f"driver == {other_drivers}")[cols]
    
    # Zamiana kierowcy bierzącego na innego (który staje się bierzącym)
    other_driver_history['driver'] = driver
    
    # Jeśli nie aktualny kierowca to parametr zamiany zero
    other_driver_history['victories'] = 0    
    
    driver_history = pd.concat([driver_history, other_driver_history])

    driver_history['color'] = colors[driver]
    
    # Sortowanie wg daty, aby poprawnie zmieniać kierowców
    driver_history.sort_values(by='date', inplace=True)
    
    # W razie konieczności zaktualizuj indeks dal uzyskania ostatniego wiersza
    driver_history.reset_index(inplace=True)
    
    # Przejście po wszystkich wierszach w celu pokazania aktualnego zwycięzscy w danym wyścigu.
 
    for index, row in driver_history.iterrows():
        if not row['victories'] and index-1 > 0:
            driver_history.loc[index, 'victories'] = driver_history.loc[index-1, 'victories']
        
    # Gotowy zbiór danych
    winners_history = pd.concat([winners_history, driver_history])  

In [235]:
# Stworzenie animacji
# Animowany wykres zwycięstwa/czas dla 5 kierowców posiadających najwiecej zwycięstw  
fig = go.Figure()

fig = px.bar(
    winners_history, 
    x='victories',
    y='driver',
    color='driver',
    color_discrete_sequence=winners_history.color.unique(),
    orientation='h',
    animation_frame="date",
    animation_group="driver",
)

# Ustawienie koloru obramowania paska postępu
fig.update_traces(dict(marker_line_width=1, marker_line_color="black"))

# Zakres osi x (liczba zwycięstw)
fig.update_layout(xaxis=dict(range=[0, 100]))

# Tytuł wykresu
fig.update_layout(title_text="Rywalizacja 5 kierowców posiadających najwiecej zwycięstw w latach 1950-2020")

# Konfiguracja animacji
fig.update_layout(
    updatemenus = [
        {
            "buttons": [
                # Play
                {
                    "args": [
                        None, 
                        {
                            "frame": {
                                "duration": 100, 
                                 "redraw": False
                            }, 
                            "fromcurrent": True,
                            "transition": {
                                "duration": 100, 
                                "easing": "linear"
                            }
                        }
                    ],
                    "label": "Start",
                    "method": "animate"
                },
                # Pause
                {
                    "args": [
                        [None], 
                        {
                            "frame": {
                                "duration": 0, 
                                "redraw": False
                            },
                            "mode": "immediate",
                            "transition": {
                                "duration": 0
                            }
                        }
                    ],
                    "label": "Pauza",
                    "method": "animate"
                }
            ]
        }
    ]
)

fig.show()

In [232]:
# Tworzenie mapy kierowców wg id
winner_drivers_ids = f1_victories_biggest_winners[['driverId', 'driver']].drop_duplicates()
winner_drivers_map = {}

for _, row in winner_drivers_ids.iterrows():
    winner_drivers_map[row['driverId']] = row['driver']

In [233]:
# Tabela Pole positions
f1_biggest_winners_poles = results.query(f"driverId == {f1_biggest_winners_ids} & grid == 1")[['driverId', 'grid']]

# Tworzenie mapy kierowców wg nazwy
f1_biggest_winners_poles['driver'] = f1_biggest_winners_poles.driverId.map(winner_drivers_map)
f1_biggest_winners_poles['color'] = f1_biggest_winners_poles.driver.map(colors)

# Skumulowana tabela zdobywców Pole position
f1_biggest_winners_poles['total_poles'] = f1_biggest_winners_poles.groupby(['driverId']).cumsum()   

# Gotowy zbiór
f1_biggest_winners_total_poles = f1_biggest_winners_poles.groupby('driver').total_poles.nlargest(1).sort_values(ascending=False).head(5)
f1_biggest_winners_total_poles = pd.DataFrame(f1_biggest_winners_total_poles).reset_index()

f1_biggest_winners_total_poles['color'] = f1_biggest_winners_total_poles.driver.map(colors)


In [234]:
# Wykres pole positions
fig = px.bar(
    f1_biggest_winners_total_poles, 
    x='driver', 
    y='total_poles',
    color='driver',
    color_discrete_sequence=f1_biggest_winners_total_poles.color
)

# Ustawienie koloru obramowania paska postępu
fig.update_traces(dict(marker_line_width=1, marker_line_color="black"))

# Tytuł wykresu
fig.update_layout(title_text="Liczba zdobytych Pole Position 5 kierowców posiadających najwiecej zwycięstw w latach 1950-2020")

fig.show()

In [209]:
# Tabela Hamiltona
hamilton = drivers.query("driverRef == 'hamilton'")

In [210]:
# Tabela kierowców w wyścigach

def get_races_by_driver_id(driver_id):
    columns = ['grid', 'position', 'raceId', 'constructorId', 'statusId']

    driver_races = results.query(f'driverId == {driver_id}')
    driver_races = driver_races[columns]

    driver_races.set_index('raceId', inplace=True)

    driver_races = driver_races.join(races.set_index('raceId')['date'])

    driver_races['is_pole'] = driver_races.grid == 1
    driver_races['is_first_place'] = driver_races.position == '1'

    driver_races.sort_values(by='date', inplace=True)

    driver_races['poles'] = driver_races.is_pole.cumsum()
    driver_races['races_won'] = driver_races.is_first_place.cumsum()

    driver_races = driver_races.set_index('constructorId').join(constructors.set_index('constructorId')['name'])
    driver_races = driver_races.rename(columns={'name': 'constructor'})
    
    driver_races = pd.merge(status, driver_races, on=['statusId', 'statusId']).sort_values(by='date')
    driver_races = driver_races.rename(columns={'status': 'race_status'})
    
    return driver_races
    
hamilton_races = get_races_by_driver_id(hamilton.driverId[0])

In [211]:
# Zespoły do wykresu
mc_laren = hamilton_races.query('constructor == "McLaren"')
mercedes = hamilton_races.query('constructor == "Mercedes"')

# Złożenie wykresów
mc_laren = pd.concat([mc_laren, mercedes.head(1)])

In [212]:
# Zdobyte Pole position
mclaren_poles  = go.Scatter(x=mc_laren.date, y=mc_laren.poles, fill='tozeroy', name="Mc Laren", marker=dict(color="#D89A8C"))
mercedes_poles = go.Scatter(x=mercedes.date, y=mercedes.poles, fill='tozeroy', name="Mercedes", marker=dict(color="#C2C2C2"))

# Wygrane wyścigi
mclaren_wons  = go.Scatter(x=mc_laren.date, y=mc_laren.races_won, fill='tozeroy', name="Mc Laren", marker=dict(color="#cb7967"), showlegend=False)
mercedes_wons = go.Scatter(x=mercedes.date, y=mercedes.races_won, fill='tozeroy', name="Mercedes", marker=dict(color="#b3b3b3"), showlegend=False)

# Rysowanie wykresu
fig = make_subplots(
    rows=2, 
    cols=1, 
    subplot_titles=("Zdobyte Pole position","Wygrane Wyścigi")
)

fig.add_trace(mclaren_poles, row=1, col=1)
fig.add_trace(mercedes_poles, row=1, col=1)

fig.add_trace(mclaren_wons, row=2, col=1)
fig.add_trace(mercedes_wons, row=2, col=1)

fig.update_layout(
    height=600,
    title_text="Hamilton w liczbach",
    title_font_size=20,
    hovermode='x',
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.95,
        xanchor="left",
        x=0.01
    ),
)

fig.update_yaxes(range=[0, 100])

_W ciągu trzynastu lat liczba zdobytych pole postion i wygranych wyścigów Hamiltona rośnie._

_W czasie gdy Lewis jeżdził dla McLarena wzrost wydaje się powolny, w Mercedesie bardziej agresywny._

_Sprawdźmy współczynnik "agresywności", w tym celu obliczymy regeresję liniową._

In [213]:
# Regresja liniowa

def linear_regression(x, y):
    x = np.array([
        time.mktime(datetime.datetime.strptime(date, "%Y-%m-%d").timetuple())
        for date in x
    ])

    y = np.array(y)

    slope, intercept, _, _, _ = sp.linregress(x,y)
    y_regression = (slope * x) + intercept
    
    return y_regression, slope

In [214]:
# Wyliczenie regresji liniowej

mc_laren_poles_y_reg, mc_laren_poles_slope = linear_regression(mc_laren.date, mc_laren.poles)
mercedes_poles_y_reg, mercedes_poles_slope = linear_regression(mercedes.date, mercedes.poles)

mc_laren_wons_y_reg, mc_laren_wons_slope = linear_regression(mc_laren.date, mc_laren.races_won)
mercedes_wons_y_reg, mercedes_wons_slope = linear_regression(mercedes.date, mercedes.races_won)

# Przebieg regresji liniowej

mc_laren_poles_linreg = go.Scatter(x=mc_laren.date, y=mc_laren_poles_y_reg, line=dict(color='#984634', dash='dash'), hoverinfo='skip')
mercedes_poles_linreg = go.Scatter(x=mercedes.date, y=mercedes_poles_y_reg, line=dict(color='#999999', dash='dash'), hoverinfo='skip')

mc_laren_wons_linreg = go.Scatter(x=mc_laren.date, y=mc_laren_wons_y_reg, line=dict(color='#984634', dash='dash'), hoverinfo='skip')
mercedes_wons_linreg = go.Scatter(x=mercedes.date, y=mercedes_wons_y_reg, line=dict(color='#999999', dash='dash'), hoverinfo='skip')

In [215]:
# Przygotowanie wykresów
fig = make_subplots(
    rows=2, 
    cols=2, 
    subplot_titles=(
        "Zdobyte Pole position w Mc Laren",
        "Zdobyte Pole position w Mercedes",
        "Wygrane wyścigi w Mc Laren",
        "Wygrane wyścigi w Mercedes",
    )
)

fig.add_trace(mc_laren_poles_linreg, row=1, col=1)
fig.add_trace(mclaren_poles, row=1, col=1)

fig.add_trace(mercedes_poles_linreg, row=1, col=2)
fig.add_trace(mercedes_poles, row=1, col=2)

fig.add_trace(mc_laren_wons_linreg, row=2, col=1)
fig.add_trace(mclaren_wons, row=2, col=1)

fig.add_trace(mercedes_wons_linreg, row=2, col=2)
fig.add_trace(mercedes_wons, row=2, col=2)

fig.update_layout(
    height=900,
    title_text="Regresja liniowa",
    title_font_size=20,
    hovermode='x',
    showlegend=False,
    legend=dict(
        yanchor="top",
        y=0.95,
        xanchor="left",
        x=0.01
    ),
)

fig.add_annotation(
    x=list(mc_laren.date)[50], 
    y=mc_laren_poles_y_reg[50],
    xref='x1', 
    yref='y1',
    text=f"Współczynnik nachylenie regresji liniowej: {mc_laren_poles_slope}",
    showarrow=True,
    bordercolor="#929191",
    borderwidth=2,
    borderpad=4,
    ay=-80,
    ax=-30,
    arrowcolor="#929191",
    arrowwidth=2
)

fig.add_annotation (
    x=list(mercedes.date)[50], 
    y=mercedes_poles_y_reg[50],
    xref='x2', 
    yref='y2',
    text=f"Współczynnik nachylenie regresji liniowej: {mercedes_poles_slope}",
    showarrow=True,
    bordercolor="#ffffff",
    borderwidth=2,
    borderpad=4,
    ay=80,
    ax=30,
    arrowcolor="#ffffff",
    arrowwidth=2
)

fig.add_annotation (
    x=list(mc_laren.date)[50], 
    y=mc_laren_wons_y_reg[50],
    xref='x3', 
    yref='y3',
    text=f"Współczynnik nachylenie regresji liniowej: {mc_laren_wons_slope}",
    showarrow=True,
    bordercolor="#929191",
    borderwidth=2,
    borderpad=4,
    ay=-80,
    ax=-30,
    arrowcolor="#929191",
    arrowwidth=2
)

fig.add_annotation (
    x=list(mercedes.date)[50], 
    y=mercedes_wons_y_reg[50],
    xref='x4', 
    yref='y4',
    text=f"Współczynnik nachylenie regresji liniowej: {mercedes_wons_slope}",
    showarrow=True,
    bordercolor="#ffffff",
    borderwidth=2,
    borderpad=4,
    ay=80,
    ax=30,
    arrowcolor="#ffffff",
    arrowwidth=2
)

fig.update_yaxes(range=[0, 100])

_Analiza otrzymanych wykresów i przebiegów linii regresji liniowej, daje nam jasny przekaz._

_Wzrost Hamiltona w Mercedesie jest prawie 3 razy szybszy niż w McLarenie. (wynika to z porównania współczynników nachulenia regresji liniowej)_

## Hamilton na tle zespołowych partnerów w McLarenie

In [216]:
hamilton_mclaren_races = hamilton_races.query("constructor == 'McLaren'")
# Przygotowanie danych dla Mc Larena

mclaren_id = constructors.query("name == 'McLaren'")['constructorId']
mclaren_id = int(mclaren_id)

mclaren_races = results.query(f"constructorId == {mclaren_id}")
mclaren_races = mclaren_races.merge(races, on='raceId')

first_hamilton_race_1 = hamilton_mclaren_races.date.min()
last_hamilton_race_1 = hamilton_mclaren_races.date.max()

hamilton_mclaren_team_mates_id = mclaren_races.query(f"'{first_hamilton_race_1}' <= date <= '{last_hamilton_race_1}' & driverId != {hamilton.driverId[0]}")
hamilton_mclaren_team_mates_id = hamilton_mclaren_team_mates_id.driverId.unique()

hamilton_mclaren_team_mates = []
    
for driver_id in hamilton_mclaren_team_mates_id:
    team_mate_races_1 = get_races_by_driver_id(driver_id).query(f"constructor == 'McLaren' & '{first_hamilton_race_1}' <= date <= '{last_hamilton_race_1}'")
    
    team_mate_races_1['driver'] = drivers.query(f"driverId == {driver_id}").driver.unique()[0]
    
    hamilton_mclaren_team_mates.append(team_mate_races_1)
    


In [219]:
def team_mate_comparisson_1(index, team_mate_color):
    team_mate_name_1 = hamilton_mclaren_team_mates[index].driver.unique()[0]
    first_team_mate_race_1 = hamilton_mclaren_team_mates[index].date.min()

   
    if index == 0:
        last_team_mate_race_1 = '2009-11-01'
     
        offset = 0
    if index == 1:
        last_team_mate_race_1 = '2007-10-21'
        offset = 15

    if index == 2:
        last_team_mate_race_1 = '2012-11-25'
        offset = 7
    
 #   hamilton_color = '#b3b3b3'  
    hamilton_color = '#b803ff'    

    # Wyścigi z partnerem z zespołu
    hamilton_mclaren_team_mate_races = hamilton_mclaren_races.query(f"'{first_team_mate_race_1}' <= date <= '{last_team_mate_race_1}'").copy()

    # Wygrane wyścigi zespołu
    hamilton_mclaren_team_mate_races['races_won'] = hamilton_mclaren_team_mate_races.is_first_place.cumsum()
    
   
    team_mate_races_wons_1 = go.Scatter(
        name=team_mate_name_1, 
        fill='tozeroy', 
        marker=dict(color=team_mate_color),
        x=hamilton_mclaren_team_mates[index].date, 
        y=hamilton_mclaren_team_mates[index].races_won-offset
    )

    hamilton_races_wons_1 = go.Scatter(
        name='Lewis Hamilton',
        fill='tozeroy',
        marker=dict(color=hamilton_color),
        x=hamilton_mclaren_team_mate_races.date, 
        y=hamilton_mclaren_team_mate_races.races_won      
    )

    # Rysowanie wykresu
    fig = make_subplots(
        rows=2, 
        cols=2, 
        specs=[
            [{"colspan": 2}, None],
            [{}, {}],
        ],
        horizontal_spacing = 0.1,
        vertical_spacing = 0.2,
        subplot_titles= (
            "Wygrane wyścigi", 
            "Rozkład zdobywanych miejsc",
            "Rozkład zdobywanych miejsc",
        )
    )

    fig.add_trace(hamilton_races_wons_1,row=1, col=1)
    fig.add_trace(team_mate_races_wons_1, row=1, col=1)

    # Klasyfikacja
    hamilton_standings_1 = go.Box(
        name='Lewis Hamilton', 
        showlegend=False, 
        marker=dict(color=hamilton_color),
        y=hamilton_mclaren_team_mate_races.position
    )

    team_mate_standings_1 = go.Box(
        name=team_mate_name_1, 
        showlegend=False, 
        marker=dict(color=team_mate_color),
        y=hamilton_mclaren_team_mates[index].position, 
    )

    fig.add_trace(hamilton_standings_1, row=2, col=1)
    fig.add_trace(team_mate_standings_1,row=2, col=2)

    fig.update_layout(
        height=700,
        margin=dict(b=10),
        title_text=f"Lewis Hamilton i {team_mate_name_1}",
        title_font_size=20,
        hovermode='x',
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.95,
            xanchor="left",
            x=0.01
        ),
    )

    fig.update_layout(    
        {    
            'yaxis': {'range': [0, 50]},
            'yaxis2':{'range': [20, 0], "nticks": 10},
            'yaxis3':{'range': [20, 0], "nticks": 10},
        }
    )
    
    # Rysowanie wykresu
    fig_race_status = make_subplots(
        rows=1, 
        cols=2, 
        specs=[
            [{}, {}]
        ],
        horizontal_spacing = 0.1,
        subplot_titles= (
            "Sposób zakończenie wyścigu",
            "Sposób zakończenie wyścigu",
        )
    )
    
    # Sposób zakończenia rywalizacji w wyścigu
    hamilton_race_status_data_1  = hamilton_mclaren_team_mate_races.race_status.value_counts().to_frame()
    team_mate_race_status_data_1 = hamilton_mclaren_team_mates[index].race_status.value_counts().to_frame()

    hamilton_race_status_data_1['percent'] = hamilton_mclaren_team_mate_races.race_status.value_counts(normalize=True)
    team_mate_race_status_data_1['percent'] = hamilton_mclaren_team_mates[index].race_status.value_counts(normalize=True)
    
    hamilton_race_status_1 = go.Bar(
        name='Lewis Hamilton', 
        orientation='h',
        showlegend=False, 
        marker=dict(color=hamilton_color),
        x=hamilton_race_status_data_1.race_status,
        y=hamilton_race_status_data_1.index,
        hoverinfo='x+y+text',
        hovertext=hamilton_race_status_data_1.percent
    )

    team_mate_race_status_1 = go.Bar(
        name=team_mate_name_1, 
        orientation='h',
        showlegend=False, 
        marker=dict(color=team_mate_color),
        x=team_mate_race_status_data_1.race_status,
        y=team_mate_race_status_data_1.index, 
        hoverinfo='x+y+text',
        hovertext=team_mate_race_status_data_1.percent
    )
    
    fig_race_status.add_trace(hamilton_race_status_1, row=1, col=1)
    fig_race_status.add_trace(team_mate_race_status_1,row=1, col=2)

    fig_race_status.update_layout(
        height=300,
        margin=dict(t=20),
        showlegend=False,
    )
    
    fig_race_status.update_layout(    
        {    
            'yaxis1':{'autorange': 'reversed'},
            'yaxis2':{'autorange': 'reversed'}
        }
    )
    
    fig.show()
    fig_race_status.show()

In [226]:
team_mate_comparisson_1(index=1, team_mate_color='#ffd700')

_Analizując rywalizację Lewis Hamiltona z Fernaddo Alonso w jego pierwszym sezonie w F1, widzimy że nieustępował pola swojemu doświadczonemu koledze._

In [228]:
team_mate_comparisson_1(index=0, team_mate_color='#0000ff')

_Heikki Kovalainen musiał uznać wyższość Hamiltona w rywalizacji z zespołowym kolegą._

In [229]:
team_mate_comparisson_1(index=2, team_mate_color='#ec6331')

_Porównując rywalizację wyczynów Lewisa Hamiltona i Jensona Buttona podczas ich wspólnej rywalizacji w McLarenie, możemy powiedzieć, że byli godnymi siebie rywalami._

## Hamilton na tle zespołowych partnerów w Mercedesie

In [164]:
hamilton_mercedes_races = hamilton_races.query("constructor == 'Mercedes'")
# Przygotowanie danych dla Mercedesa

mercedes_id = constructors.query("name == 'Mercedes'")['constructorId']
mercedes_id = int(mercedes_id)

mercedes_races = results.query(f"constructorId == {mercedes_id}")
mercedes_races = mercedes_races.merge(races, on='raceId')

first_hamilton_race = hamilton_mercedes_races.date.min()

hamilton_mercedes_team_mates_id = mercedes_races.query(f"date >= '{first_hamilton_race}' & driverId != {hamilton.driverId[0]}")
hamilton_mercedes_team_mates_id = hamilton_mercedes_team_mates_id.driverId.unique()

hamilton_mercedes_team_mates = []
    
for driver_id in hamilton_mercedes_team_mates_id:
    team_mate_races = get_races_by_driver_id(driver_id).query(f"constructor == 'Mercedes' & date >= '{first_hamilton_race}'")
    
    team_mate_races['driver'] = drivers.query(f"driverId == {driver_id}").driver.unique()[0]
    
    hamilton_mercedes_team_mates.append(team_mate_races)
    


In [221]:
def team_mate_comparisson(index, team_mate_color):
    team_mate_name = hamilton_mercedes_team_mates[index].driver.unique()[0]

    last_team_mate_race = hamilton_mercedes_team_mates[index].date.max()
    first_team_mate_race = hamilton_mercedes_team_mates[index].date.min()
    
    hamilton_color = '#b803ff'    

    # Wyścigi z partnerem z zespołu
    hamilton_mercedes_team_mate_races = hamilton_mercedes_races.query(f"'{first_team_mate_race}' <= date <= '{last_team_mate_race}'").copy()

    # Wygrane wyścigi zespołu
    hamilton_mercedes_team_mate_races['races_won'] = hamilton_mercedes_team_mate_races.is_first_place.cumsum()

    team_mate_races_wons = go.Scatter(
        name=team_mate_name, 
        fill='tozeroy', 
        marker=dict(color=team_mate_color),
        x=hamilton_mercedes_team_mates[index].date, 
        y=hamilton_mercedes_team_mates[index].races_won 
    )

    hamilton_races_wons = go.Scatter(
        name='Lewis Hamilton',
        fill='tozeroy',
        marker=dict(color=hamilton_color),
        x=hamilton_mercedes_team_mate_races.date, 
        y=hamilton_mercedes_team_mate_races.races_won      
    )

    # Rysowanie wykresu
    fig = make_subplots(
        rows=2, 
        cols=2, 
        specs=[
            [{"colspan": 2}, None],
            [{}, {}],
        ],
        horizontal_spacing = 0.1,
        vertical_spacing = 0.2,
        subplot_titles= (
            "Wygrane wyścigi", 
            "Rozkład zdobywanych miejsc",
            "Rozkład zdobywanych miejsc",
        )
    )

    fig.add_trace(hamilton_races_wons,row=1, col=1)
    fig.add_trace(team_mate_races_wons, row=1, col=1)

    # Klasyfikacja
    hamilton_standings = go.Box(
        name='Lewis Hamilton', 
        showlegend=False, 
        marker=dict(color=hamilton_color),
        y=hamilton_mercedes_team_mate_races.position
    )

    team_mate_standings = go.Box(
        name=team_mate_name, 
        showlegend=False, 
        marker=dict(color=team_mate_color),
        y=hamilton_mercedes_team_mates[index].position, 
    )

    fig.add_trace(hamilton_standings, row=2, col=1)
    fig.add_trace(team_mate_standings,row=2, col=2)

    fig.update_layout(
        height=700,
        margin=dict(b=10),
        title_text=f"Lewis Hamilton i {team_mate_name}",
        title_font_size=20,
        hovermode='x',
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.95,
            xanchor="left",
            x=0.01
        ),
    )

    fig.update_layout(    
        {    
            'yaxis': {'range': [0, 50]},
            'yaxis2':{'range': [20, 0], "nticks": 10},
            'yaxis3':{'range': [20, 0], "nticks": 10},
        }
    )
    
    # Rysowanie wykresu
    fig_race_status = make_subplots(
        rows=1, 
        cols=2, 
        specs=[
            [{}, {}]
        ],
        horizontal_spacing = 0.1,
        subplot_titles= (
            "Sposób zakończenie wyścigu",
            "Sposób zakończenie wyścigu",
        )
    )
    
    # Sposób zakończenia rywalizacji w wyścigu
    hamilton_race_status_data  = hamilton_mercedes_team_mate_races.race_status.value_counts().to_frame()
    team_mate_race_status_data = hamilton_mercedes_team_mates[index].race_status.value_counts().to_frame()

    hamilton_race_status_data['percent'] = hamilton_mercedes_team_mate_races.race_status.value_counts(normalize=True)
    team_mate_race_status_data['percent'] = hamilton_mercedes_team_mates[index].race_status.value_counts(normalize=True)
    
    hamilton_race_status = go.Bar(
        name='Lewis Hamilton', 
        orientation='h',
        showlegend=False, 
        marker=dict(color=hamilton_color),
        x=hamilton_race_status_data.race_status,
        y=hamilton_race_status_data.index,
        hoverinfo='x+y+text',
        hovertext=hamilton_race_status_data.percent
    )

    team_mate_race_status = go.Bar(
        name=team_mate_name, 
        orientation='h',
        showlegend=False, 
        marker=dict(color=team_mate_color),
        x=team_mate_race_status_data.race_status,
        y=team_mate_race_status_data.index, 
        hoverinfo='x+y+text',
        hovertext=team_mate_race_status_data.percent
    )
    
    fig_race_status.add_trace(hamilton_race_status, row=1, col=1)
    fig_race_status.add_trace(team_mate_race_status,row=1, col=2)

    fig_race_status.update_layout(
        height=300,
        margin=dict(t=20),
        showlegend=False,
    )
    
    fig_race_status.update_layout(    
        {    
            'yaxis1':{'autorange': 'reversed'},
            'yaxis2':{'autorange': 'reversed'}
        }
    )
    
    fig.show()
    fig_race_status.show()

In [224]:
team_mate_comparisson(index=0, team_mate_color='#0e0000')

_Na początku rywalizacji z Nico Rosbergiem Hamilton musiał uznac wyższość swojego partnetra z zespołu, ale już w końcówce drugiego sezonu mial wiecej zwycięst od Rosberga. Ich rywalizacja zakończyła się wynikem 32 do 23 dla Lewisa._

In [225]:
team_mate_comparisson(index=1, team_mate_color='#6ed6d1')

_Od sezonu 2017 partnerem brytyjczyka jest Valtteri Bottas. Przewaga Hamiltona jest przytłaczająca, nie oznacza to że Bottas jest słabym kierowcą (wicemistrz świata w 2019 i 2020), tylko że Lewis jest coraz lepszy._

Analizując wyniki Hamiltona z ostatnich czterech sezonów możemy być pełni podziwu:
* ukończył 97% procent wyścigów
* najniższe miejsce to 9
* w każdym ukończonym wyścigu zdobył punkty (punkty zdobywa pierwszych 10 miejsc).  

Ogólem zdobył:
* Mistrzoswo świata w 2008, 2014, 2015, 2017, 2018, 2019, 2020
* Zdobytych Pole position: 98
* Wygrane wyścigi Gran Prix: 95